# 01. Car Scrapper

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt

from time import sleep
from random import randint

In [2]:
# Links for the different categories
small_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=1&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
sedan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=6&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
familiar_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1& cy=E&body=5&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
minivan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
coupe_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=3&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
cabrio_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=2&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
suv_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=4&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
van_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=13&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"



In [3]:
#Defining columns for small cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

The colour is not available for srapping as we cannot see the field in the main search, so the techical solution is to include it on the website link when setting coding the webpage to be scrapped as if we include &bcol=10& on the path we are extracting cars for colour 10 in this case

In [4]:
#example of the colour code in the link:
"https://www.autoscout24.es/lst/?sort=age&desc=1&bcol=1&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=1&cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"

'https://www.autoscout24.es/lst/?sort=age&desc=1&bcol=1&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=1&cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&'

In [5]:
colours = np.array([1,2,3,4,5,6,7,10,11,12,13,14,15,16])
colours

array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16])

In [6]:
colours_dictionary = {'1': 'beige', 
                      '2': 'blue', 
                      '3': 'brown', 
                      '4': 'bronze', 
                      '5': 'yellow', 
                      '6': 'grey', 
                      '7': 'green',
                      '10': 'red',
                      '11': 'black', 
                      '12': 'silver', 
                      '13': 'burgundy', 
                      '14': 'white', 
                      '15': 'orange', 
                      '16': 'gold'}

In [7]:
# Analysing urls for the diffenret pages:
# url_pag1 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag2 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=2&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag3 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=3&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"

# Max page per url = 20 (start: page 1, stop: page 21 -1, step: 1 by 1 as the url changes from page=n to page=n+1 from one page to the following

all_pages = np.arange(1,21,1)

In [8]:
#Scapping small cars

for car_colours in colours:
    for small_cars_page in all_pages:

        small_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(small_cars_page) + "&cy=E&body=1&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        small_cars_soup = BeautifulSoup(small_cars_page.text,"lxml")
        small_cars_ads = small_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in small_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [9]:
#Creating the Small_Cars_DataFrame
small_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Small",
                        "Doors" : "5",
                        "Colour": colour })


In [10]:
small_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Fiat Tipo,1.4 Easy,\n06/2017\n,\n24.000 km\n,\n70 kW (95 CV)\n,\nGasolina\n,\nManual\n,"\n6,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,43001 TARRAGONA,"\n€ 10.900,-\n",Small,5,1
1,Volvo V40,T2 Kinetic,\n02/2018\n,\n15.693 km\n,\n90 kW (122 CV)\n,\nGasolina\n,\nManual\n,"\n5,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,50013 ZARAGOZA,"\n€ 20.990,-\nIVA deducible\n",Small,5,1
2,Volvo V40,D2 Momentum Aut. 115,\n05/2014\n,\n113.800 km\n,\n84 kW (114 CV)\n,\nDiésel\n,\nAutomático\n,"\n3,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,18230 ALJARAFE,"\n€ 11.990,-\n",Small,5,1
3,SEAT Cordoba,1.9TDi Top,\n03/2005\n,\n88.800 km\n,\n74 kW (101 CV)\n,\nDiésel\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28970 HUMANES DE MADRID,"\n€ 5.990,-\n",Small,5,1
4,Renault Clio,1.2 TCE Expression,\n08/2009\n,\n64.000 km\n,\n74 kW (101 CV)\n,\nGasolina\n,\nManual\n,"\n5,8 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,8225 Terrasa,"\n€ 6.899,-\n",Small,5,1


In [11]:
#Defining columns for sedan cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [12]:
#Scapping sedan cars

for car_colours in colours:
    for sedan_cars_page in all_pages:

        sedan_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(sedan_cars_page) + "&cy=E&body=6&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        sedan_cars_soup = BeautifulSoup(sedan_cars_page.text,"lxml")
        sedan_cars_ads = sedan_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in sedan_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [13]:
#Creating the Sedan_Cars_DataFrame
sedan_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Sedan",
                        "Doors" : "5",
                        "Colour" : colour })

In [14]:
sedan_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Chevrolet Captiva,2.2 VCDI 16V 7 PLAZAS AWD LTZ,\n10/2011\n,\n86.000 km\n,\n135 kW (184 CV)\n,\nDiésel\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,22004 HUESCA,"\n€ 14.500,-\nIVA deducible\n",Sedan,5,1
1,SEAT Ibiza,Berlina Manual de 5 Puertas,\n03/2016\n,\n54.000 km\n,\n66 kW (90 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,38639 ARONA,"\n€ 9.900,-\n",Sedan,5,1
2,Rolls-Royce Silver Spirit,III,\n04/1997\n,\n24.000 km\n,\n165 kW (224 CV)\n,\nGasolina\n,\nAutomático\n,\n0 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,29651 MIJAS,"\n€ 49.000,-\n",Sedan,5,1
3,Mercedes-Benz A 200,200d AMG AUT150cv,\n10/2019\n,\n5.500 km\n,\n110 kW (150 CV)\n,\nDiésel\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,03203 ELCHE,"\n€ 29.999,-\n",Sedan,5,1
4,Peugeot 508,1.6e-HDI BlueLion Business Line CMP,\n06/2011\n,\n183.000 km\n,\n82 kW (111 CV)\n,\nDiésel\n,\nAutomático\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,20013 DONOSTIA-SAN SEBASTIÁN,"\n€ 7.500,-\n",Sedan,5,1


In [15]:
#Defining columns for familiar cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [16]:
#Scapping familiar cars

for car_colours in colours:
    for familiar_cars_page in all_pages:

        familiar_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(familiar_cars_page) + "&cy=E&body=5&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        familiar_cars_soup = BeautifulSoup(familiar_cars_page.text,"lxml")
        familiar_cars_ads = familiar_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in familiar_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [17]:
#Creating the Familiar_Cars_DataFrame
familiar_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Wagon",
                        "Doors" : "5",
                        "Colour" : colour })

In [18]:
familiar_cars_df

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Volkswagen Golf,Variant 1.6TDI Advance,\n04/2018\n,\n92.000 km\n,\n85 kW (116 CV)\n,\nDiésel\n,\nManual\n,"\n4,1 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28939 Arroyomolinos,"\n€ 14.449,-\n",Wagon,5,1
1,Audi A6,Avant 2.0TDI,\n03/2005\n,\n219.879 km\n,\n103 kW (140 CV)\n,\nDiésel\n,\nManual\n,"\n6,2 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,04230 ALMERIA,"\n€ 6.490,-\n",Wagon,5,1
2,SEAT Leon,ST 2.0TDI CR S,\n05/2015\n,\n91.283 km\n,\n110 kW (150 CV)\n,\nDiésel\n,\nManual\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n112 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 11.400,-\n",Wagon,5,1
3,Peugeot 207,SW 1.4i Confort,\n04/2009\n,\n165.260 km\n,\n55 kW (75 CV)\n,\nGasolina\n,\nManual\n,"\n6,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08720 Vilafranca del Penedes,"\n€ 4.600,-\n",Wagon,5,1
4,Volvo V70,D5 Momentum,\n07/2008\n,\n260.000 km\n,\n136 kW (185 CV)\n,\nDiésel\n,\nManual\n,"\n6,7 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,48810 ALONSOTEGI,"\n€ 7.490,-\n",Wagon,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,Volvo V50,2.0D DPF Momentum,\n04/2007\n,\n204.000 km\n,\n100 kW (136 CV)\n,\nDiésel\n,\nManual\n,"\n5,7 l/100 km (mixto)\nPuede obtenerse inform...",\n153 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 4.000,-\n",Wagon,5,16
2869,Volkswagen Transporter,Mixto 1.9TDI 104,\n03/2006\n,\n314.000 km\n,\n76 kW (103 CV)\n,\nDiésel\n,\nManual\n,"\n7,5 l/100 km (mixto)\nPuede obtenerse inform...",\n203 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 10.000,-\n",Wagon,5,16
2870,Citroen C5,Tourer 1.6HDI Millenium 115,\n06/2013\n,\n212.000 km\n,\n84 kW (114 CV)\n,\nDiésel\n,\nAutomático\n,\n5 l/100 km (mixto)\nPuede obtenerse informac...,\n129 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 8.700,-\nIVA deducible\n",Wagon,5,16
2871,Audi A4,Avant 2.0TDI Sport edition S tronic 110kW,\n12/2015\n,\n200.000 km\n,\n110 kW (150 CV)\n,\nDiésel\n,\nAutomático\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n116 g/km (mixto)\nPuede obtenerse informació...,\n1 Propietario\n,,"\n€ 12.700,-\nIVA deducible\n",Wagon,5,16


In [19]:
#Defining columns for minivan cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [20]:
#Scapping minivans cars

for car_colours in colours:
    for minivan_cars_page in all_pages:

        minivan_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(minivan_cars_page) + "&cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        minivan_cars_soup = BeautifulSoup(minivan_cars_page.text,"lxml")
        minivan_cars_ads = minivan_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in minivan_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [21]:
#Creating the Minivan_Cars_DataFrame
minivan_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Minivan",
                        "Doors" : "5",
                        "Colour" : colour })

In [22]:
minivan_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Citroen C4,Picasso 1.6VTi Premier,\n06/2009\n,\n160.000 km\n,\n88 kW (120 CV)\n,\nGasolina\n,\nManual\n,"\n7,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,48950 ERANDIO,"\n€ 5.990,-\n",Minivan,5,1
1,Renault Scenic,1.6 Dynamique,\n04/2006\n,\n92.400 km\n,\n82 kW (111 CV)\n,\nGasolina\n,\nManual\n,"\n7,6 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28970 HUMANES DE MADRID,"\n€ 4.990,-\n",Minivan,5,1
2,Opel Zafira,1.7CDTi Essentia,\n02/2010\n,\n202.699 km\n,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,"\n5,3 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28232 LAS ROZAS,"\n€ 5.290,-\n",Minivan,5,1
3,Peugeot 3008,1.6HDI FAP Active 110,\n12/2011\n,\n103.900 km\n,\n82 kW (111 CV)\n,\nDiésel\n,\nManual\n,\n5 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,48510 Trapagaran,"\n€ 9.900,-\n",Minivan,5,1
4,SEAT Altea XL,1.9TDI Reference,\n01/2010\n,\n233.209 km\n,\n77 kW (105 CV)\n,\nDiésel\n,\nManual\n,"\n5,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,46930 QUART DE POBLET,"\n€ 3.400,-\n",Minivan,5,1


In [23]:
#Defining columns for coupe cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [25]:
#Scapping coupe cars

for car_colours in colours:
    for coupe_cars_page in all_pages:

        coupe_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(coupe_cars_page) + "&cy=E&body=3&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        coupe_cars_soup = BeautifulSoup(coupe_cars_page.text,"lxml")
        coupe_cars_ads = coupe_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in coupe_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [26]:
#Creating the Coupe_Cars_DataFrame
coupe_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Coupe",
                        "Doors" : "3",
                        "Colour" : colour})

In [27]:
coupe_cars_df

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Audi A5,Coupé 3.0TDI quattro,\n11/2007\n,\n235.000 km\n,\n177 kW (241 CV)\n,\nDiésel\n,\nManual\n,"\n6,6 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,20013 DONOSTIA-SAN SEBASTIÁN,"\n€ 10.900,-\n",Coupe,3,1
1,Mercedes-Benz C 180,Sportcoupé 180K Indianapolis,\n07/2004\n,\n112.000 km\n,\n105 kW (143 CV)\n,\nGasolina\n,\nManual\n,"\n9,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08227 TERRASSA,"\n€ 5.900,-\n",Coupe,3,1
2,Mercedes-Benz CLK 200,Kompressor,\n07/2005\n,\n115.000 km\n,\n120 kW (163 CV)\n,\nGasolina\n,\nAutomático\n,"\n8,3 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,19171 CABANILLAS DEL CAMPO,"\n€ 6.995,-\n",Coupe,3,1
3,Volkswagen Escarabajo,1300,\n07/1970\n,\n90.200 km\n,\n29 kW (39 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n1 Propietario\n,,"\n€ 7.500,-\n",Coupe,3,1
4,Chrysler Imperial,,\n01/1960\n,\n120.000 km\n,\n261 kW (355 CV)\n,\nGasolina\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,,"\n€ 28.000,-\n",Coupe,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,Mercedes-Benz CLS 320,320CDI Aut.,\n12/2007\n,\n285.000 km\n,\n165 kW (224 CV)\n,\nDiésel\n,\nAutomático\n,"\n7,6 l/100 km (mixto)\nPuede obtenerse inform...",\n200 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 9.000,-\n",Coupe,3,16
4620,BMW M4,A,\n05/2017\n,\n29.000 km\n,\n317 kW (431 CV)\n,\nGasolina\n,\nAutomático\n,"\n8,3 l/100 km (mixto)\nPuede obtenerse inform...",\n194 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 66.500,-\nIVA deducible\n",Coupe,3,16
4621,Volvo C70,2.5 T / 2.4 T,\n12/2000\n,\n170.000 km\n,\n143 kW (194 CV)\n,\nGasolina\n,\nManual\n,"\n10,1 l/100 km (mixto)\nPuede obtenerse infor...",\n- (g/km)\n,\n- (Propietarios)\n,,"\n€ 2.500,-\n",Coupe,3,16
4622,BMW M3,,\n11/2002\n,\n225.000 km\n,\n252 kW (343 CV)\n,\nGasolina\n,\nManual\n,"\n11,9 l/100 km (mixto)\nPuede obtenerse infor...",\n- (g/km)\n,\n- (Propietarios)\n,,"\n€ 22.000,-\n",Coupe,3,16


In [28]:
#Defining columns for cabrio cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [29]:
#Scapping cabrio cars

for car_colours in colours:
    for cabrio_cars_page in all_pages:

        cabrio_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(cabrio_cars_page) + "&cy=E&body=2&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        cabrio_cars_soup = BeautifulSoup(cabrio_cars_page.text,"lxml")
        cabrio_cars_ads = cabrio_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in cabrio_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [30]:
#Creating the Cabrio_Cars_DataFrame
cabrio_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Convertible",
                        "Doors" : "3",
                        "Colour" : colour })

In [31]:
cabrio_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,BMW 335,335i Cabrio Aut.,\n09/2007\n,\n73.000 km\n,\n225 kW (306 CV)\n,\nGasolina\n,\nAutomático\n,"\n9,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,8120 La Llagosta,"\n€ 18.500,-\n",Convertible,3,1
1,Peugeot 207,CC 1.6 VTi,\n03/2008\n,\n165.898 km\n,\n88 kW (120 CV)\n,\nGasolina\n,\nManual\n,"\n6,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,46930 QUART DE POBLET,"\n€ 3.990,-\n",Convertible,3,1
2,Morgan Plus 4,plus 4,\n06/1956\n,\n34.257 km\n,\n66 kW (90 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n2 Propietarios\n,,"\n€ 35.000,-\n",Convertible,3,1
3,MINI Cooper S Cabrio,Mini Aut.,\n12/2011\n,\n70.000 km\n,\n135 kW (184 CV)\n,\nGasolina\n,\nAutomático\n,"\n6,6 l/100 km (mixto)\nPuede obtenerse inform...",\n153 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 20.000,-\n",Convertible,3,1
4,Renault Megane,Mégane C.C. 1.9DCI Confort Dynamique,\n10/2005\n,\n225.000 km\n,\n88 kW (120 CV)\n,\nDiésel\n,\nManual\n,"\n5,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,,"\n€ 3.300,-\n",Convertible,3,1


In [32]:
#Defining columns for suv cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [33]:
#Scapping suv cars

for car_colours in colours:
    for suv_cars_page in all_pages:

        suv_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(suv_cars_page) + "&cy=E&body=4&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        suv_cars_soup = BeautifulSoup(suv_cars_page.text,"lxml")
        suv_cars_ads = suv_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in suv_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [34]:
#Creating the Suv_Cars_DataFrame
suv_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Suv",
                        "Doors" : "5",
                        "Colour" : colour })

In [35]:
suv_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Land Rover Range Rover Velar,2.0D HSE 4WD Aut. 240,\n01/2019\n,\n70.000 km\n,\n177 kW (241 CV)\n,\nDiésel\n,\nAutomático\n,"\n5,8 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,49025 ZAMORA,"\n€ 47.990,-\nIVA deducible\n",Suv,5,1
1,Hyundai Tucson,1.7CRDI BD Klass Nav 4x2,\n01/2018\n,\n102.000 km\n,\n84 kW (114 CV)\n,\nDiésel\n,\nManual\n,"\n4,6 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28919 Leganés,"\n€ 20.200,-\n",Suv,5,1
2,Lexus RX 400,H Luxury,\n02/2007\n,\n215.000 km\n,\n200 kW (272 CV)\n,\nElectro/Gasolina\n,\nAutomático\n,"\n8,1 l/100 km (mixto)\nPuede obtenerse inform...",\n192 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 9.500,-\n",Suv,5,1
3,Jeep Wrangler,2.8CRD Rubicon Recon Aut.,\n07/2018\n,\n27.000 km\n,\n147 kW (200 CV)\n,\nDiésel\n,\nAutomático\n,\n9 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,31110 Noain,"\n€ 42.000,-\n",Suv,5,1
4,Kia Sorento,2.5CRDI EX,\n01/2004\n,\n250.000 km\n,\n103 kW (140 CV)\n,\nDiésel\n,\nManual\n,"\n8,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,06200 ALMENDRALEJO,"\n€ 4.100,-\n",Suv,5,1


In [36]:
#Defining columns for Van cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []

In [37]:
#Scapping Van cars

for car_colours in colours:
    for van_cars_page in all_pages:

        van_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(van_cars_page) + "&cy=E&body=13&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
        van_cars_soup = BeautifulSoup(van_cars_page.text,"lxml")
        van_cars_ads = van_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

        sleep(randint(2,10))

        for element in van_cars_ads:
            try:
                Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
            except:
                Title = ""
            title.append(Title)

            try:
                Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
            except:
                Version = ""
            version.append(Version)
            try:
                Year = element.find('li', attrs={'data-type':'first-registration'}).text
            except:
                Year = ""
            year.append(Year)
            try:
                Kms = element.find('li', attrs={"data-type" : "mileage"}).text
            except:
                Kms = ""
            kms.append(Kms)

            try:
                Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
            except:
                Gear = ""
            gear.append(Gear)

            try:
                Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
            except:
                Fuel = ""
            fuel.append(Fuel)

            try:
                Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
            except:
                Fuel_cons = ""
            fuel_cons.append(Fuel_cons)

            try:
                Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
            except:
                Co2 = ""
            co2.append(Co2)

            try:
                Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
            except:
                Prev_owners = ""
            prev_owners.append(Prev_owners)
            try:
                Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
            except:
                Price = ""
            price.append(Price)

            try:
                Hp = element.find('li', text =re.compile(r'CV')).text
            except:
                Hp = ""
            hp.append(Hp)

            try:
                Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
            except:
                Location = ""
            location.append(Location)
            
            try:
                Car_colours = car_colours
                
            except :
                Car_colours = ""
            colour.append(Car_colours)

In [38]:
#Creating the Van_Cars_DataFrame
van_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Category" : "Van",
                        "Doors" : "3",
                        "Colour" : colour})

In [39]:
van_cars_df.head()

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Peugeot,Partner Tepee 1.6BlueHDI Outdoor 100,,,\n73 kW (99 CV)\n,\nDiésel\n,\nManual\n,,\n- (g/km)\n,,22004 HUESCA,"\n€ 16.400,-\nIVA deducible\n",Van,3,1
1,Volkswagen,Transporter Kombi 2.5TDI 130,,,\n96 kW (131 CV)\n,\nDiésel\n,\nManual\n,,\n- (g/km)\n,,46988 Paterna,"\n€ 13.990,-\n",Van,3,1
2,Volkswagen,Caravelle 2.0TDI BMT Origin Batalla Corta 81kW,,,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,,\n- (g/km)\n,,8720 Vilafranca Del PenedÉs,"\n€ 40.899,-\n",Van,3,1
3,Renault,Traf. 2.0dCi Generation Privilege 115,,,\n84 kW (114 CV)\n,\nDiésel\n,\nManual\n,,\n- (g/km)\n,,01013 VITORIA - GASTEIZ,"\n€ 19.800,-\n",Van,3,1
4,Renault,Traf. 2.0dCi Generation Privilege 115,,,\n84 kW (114 CV)\n,\nDiésel\n,\nManual\n,,\n- (g/km)\n,,20180 OIARTZUN,"\n€ 19.800,-\n",Van,3,1


In [40]:
#Mergin all DataFrames 
all_cars_df = pd.concat([small_cars_df,
                         sedan_cars_df,
                         familiar_cars_df,
                         minivan_cars_df,
                         coupe_cars_df,
                         cabrio_cars_df,
                         suv_cars_df,
                         van_cars_df], axis = 0)
#Reset_index
all_cars_df = all_cars_df.reset_index()

In [41]:
all_cars_df.drop(["index"],axis = 1)

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Category,Doors,Colour
0,Fiat Tipo,1.4 Easy,\n06/2017\n,\n24.000 km\n,\n70 kW (95 CV)\n,\nGasolina\n,\nManual\n,"\n6,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,43001 TARRAGONA,"\n€ 10.900,-\n",Small,5,1
1,Volvo V40,T2 Kinetic,\n02/2018\n,\n15.693 km\n,\n90 kW (122 CV)\n,\nGasolina\n,\nManual\n,"\n5,5 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,50013 ZARAGOZA,"\n€ 20.990,-\nIVA deducible\n",Small,5,1
2,Volvo V40,D2 Momentum Aut. 115,\n05/2014\n,\n113.800 km\n,\n84 kW (114 CV)\n,\nDiésel\n,\nAutomático\n,"\n3,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,18230 ALJARAFE,"\n€ 11.990,-\n",Small,5,1
3,SEAT Cordoba,1.9TDi Top,\n03/2005\n,\n88.800 km\n,\n74 kW (101 CV)\n,\nDiésel\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28970 HUMANES DE MADRID,"\n€ 5.990,-\n",Small,5,1
4,Renault Clio,1.2 TCE Expression,\n08/2009\n,\n64.000 km\n,\n74 kW (101 CV)\n,\nGasolina\n,\nManual\n,"\n5,8 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,8225 Terrasa,"\n€ 6.899,-\n",Small,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,Renault Master,Fg. dCi 95kW T L3H3 3500,\n06/2018\n,\n43.755 km\n,\n95 kW (129 CV)\n,\nDiésel\n,\nManual\n,"\n8,2 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28320 PINTO,"\n€ 24.990,-\n",Van,3,15
26582,Ford,RANGER 2.0 TDCI 157KW 4X4 DOB CAB AT WILDTRACK,,,\n157 kW (213 CV)\n,\nDiésel\n,\nAutomático\n,,\n- (g/km)\n,,45600 TALAVERA DE LA REINA,"\n€ 28.650,-\nIVA deducible\n",Van,3,15
26583,Mitsubishi L200,2.2DI-D Doble Cabina 6AT Kaiteki,\n06/2020\n,\n1 km\n,\n110 kW (150 CV)\n,\nDiésel\n,\nAutomático\n,"\n7,8 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,27003 LUGO,"\n€ 35.500,-\n",Van,3,15
26584,Ford Transit Connect,FT 230L TDCi 110,\n09/2007\n,\n266.600 km\n,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,"\n6,3 l/100 km (mixto)\nPuede obtenerse inform...",\n172 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,,"\n€ 3.400,-\n",Van,3,16


In [42]:
all_cars_df.to_csv('/users/alfon/Desktop/Master/TFM/Csv/cars_raw_colour.csv', header=True, index=False)